In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import ltrim,rtrim,trim,col
from pyspark.sql.functions import *
from pyspark import SparkContext
spark = SparkSession.builder.appName("MyFirstCSV").getOrCreate()


In [ ]:
#CREATE DATAFRAME FROM CSV FILE
oij_df = spark.read.csv( path="OIJ.csv", sep=";", header=True,quote='"',inferSchema=True,)
inec_df = spark.read.csv( path="INEC.csv", sep=";", header=True,quote='"',inferSchema=True,)

In [ ]:
def generate_new_columns(df):
    columns = ["Provincia", "Canton", "Distrito","Tasa neta de participación", "Porcentaje de población económicamente inactiva", "Relación de dependencia económica"]
    new_df = spark.createDataFrame(data =[("","","","","","")], schema = columns)
    Provincia =''
    Canton = ''
    Distrito = ''
    counter = 0
    counter2 = 0
    for row in df.collect():
        if row['Provincia, cantón y distrito'] == None:
            counter += 1
            counter2 += 1
            if counter == 4: 
                counter = 2  

            if counter2 == 2:
                counter = 1
            continue
        if counter == 1:
            Provincia = row[0]
        if counter == 2:
            Canton = row[0]
        if counter == 3:
            Distrito = row[0]
            NewRow = (Provincia, Canton, Distrito, "", "", "")
            new_df = new_df.union(spark.createDataFrame(data =[NewRow], schema = columns))
        counter2 = 0
    
    return new_df

inec_df = generate_new_columns(inec_df)

In [ ]:
#Function to remove spaces from the beginning and end of the string
def remove_spaces(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito':
            df = df.withColumn(col, trim(col))
    return df

#Function to parser the string to lowercase
def to_lower_case(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito':
            df = df.withColumn(col, lower(col))
    return df


oij_df = remove_spaces(oij_df)
inec_df = remove_spaces(inec_df)

oij_df = to_lower_case(oij_df)
inec_df = to_lower_case(inec_df)


In [8]:

def find_non_matches(df1,df2):
    non_matches = []
    is_match = False
    non_match = ''

    for row in df1.collect():
        for row2 in df2.collect():
            if row['Provincia'] == row2['Provincia'] and row['Canton'] == row2['Canton'] and row['Distrito'] == row2['Distrito']:
                is_match = True
                break
        if is_match == False:
            non_match = row['Provincia'] 

            non_matches.append(non_match)
        is_match = False
    return non_matches

non_matches = find_non_matches(oij_df,inec_df)

for value in non_matches:
    print(value)

22/11/25 14:07:30 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB


22/11/25 14:08:29 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB


22/11/25 14:09:39 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB


In [ ]:
#Function to replace the accents in column Provincia, cantón y distrito in inec_df
def replace_accents(df):
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'á', 'a'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'é', 'e'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'í', 'i'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ó', 'o'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ú', 'u'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ñ', 'n'))
    return df

inec_df = replace_accents(inec_df)
nonMatches = find_non_matches(oij_df,inec_df)
print(nonMatches)


In [ ]:
#Fucntion to separate column Provincia, cantón y distrito in inec_df in three columns


In [ ]:
def generate_new_columns(df):
    columns = ["Provincia1", "Canton1", "Distrito1","Tasa neta de participación", "Porcentaje de población económicamente inactiva", "Relación de dependencia económica"]
    new_df = spark.createDataFrame(data =[("","","","","","")], schema = columns)
    Provincia =''
    Canton = ''
    Distrito = ''
    counter = 0
    counter2 = 0
    for row in df.collect():
        if row['Provincia, cantón y distrito'] == None:
            counter += 1
            counter2 += 1
            if counter == 4: 
                counter = 2  

            if counter2 == 2:
                counter = 1
            continue
        if counter == 1:
            Provincia = row[0]
        if counter == 2:
            Canton = row[0]
        if counter == 3:
            Distrito = row[0]
            NewRow = (Provincia, Canton, Distrito, "", "", "")
            new_df = new_df.union(spark.createDataFrame(data =[NewRow], schema = columns))
        counter2 = 0
    
    return new_df

new_df = generate_new_columns(inec_df)

In [ ]:
#Function to create a new dataframe with inec_df and oij_df when provincia, canton and distrito are equals
def join_dataframes(df1,df2):
    #df = df1.join(df2, (df1.Provincia == df2.Provincia) & (df1.Canton == df2.Canton) & (df1.Distrito == df2.Distrito), 'inner')
    df = df1.join(df2, (df1.Provincia == df2.Provincia1) & (df1.Canton == df2.Canton1), 'inner')
    return df

df = join_dataframes(oij_df,new_df)
df.show(25)


In [ ]:
#Create sql table from dataframe
#df.createOrReplaceTempView("df")

#Save the dataframe in a postgresql database
df.write.jdbc(url="jdbc:postgresql://localhost:5432/etl", table="etl", mode="overwrite", properties={"user": "postgres", "password": "None"})


In [ ]:
#Save the dataframe in csv file
df.write.csv(path="output.csv", sep=",", header=True,quote='"',mode="overwrite")
df.show(5)
